In [2]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

In [3]:
load_dotenv()
key = os.getenv('OPENAI_API_KEY')

In [5]:
llm = ChatOpenAI(openai_api_key=key, model_name="gpt-3.5-turbo", temperature=0.5)

e:\Personal\LLM\langchain_mcqgen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker GIven the above text, it is your job to\
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questionsto be conforming the text as well.
Make sure to format your response like RESPONSE_JSON and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer',
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer',
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer',
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer',
    },
    "5": {
        "mcq": "multiple choice question",
        "options": {
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer',
    }
}

In [17]:
quiz_generation_prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=['text','number','subject','tone','response_json']
)

In [12]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key='quiz',
    verbose=True
)

e:\Personal\LLM\langchain_mcqgen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
# a second task will be setup to review the questions framed

TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students,\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the analysis.\
If the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changes and change the tone such that it perfectly fits the student abilities.
Quiz MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    template=TEMPLATE2,
    input_variables=['subject','quiz']
)

In [15]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key='review',
    verbose=True
)

In [18]:
# combining both the chains using SequentialChain
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
)

In [20]:
with open(r"E:\Personal\LLM\langchain_mcqgen\data.txt", "r") as f:
    TEXT = f.read()

print(TEXT)

The Reserve Bank of India, abbreviated as RBI, is India's central bank and regulatory body responsible for regulation of the Indian banking system. Owned by the Ministry of Finance, Government of India, it is responsible for the control, issue and maintaining supply of the Indian rupee. It also manages the country's main payment systems and works to promote its economic development. Bharatiya Reserve Bank Note Mudran (BRBNM) is a specialised division of RBI through which it prints and mints Indian currency notes (INR) in two of its currency printing presses located in Mysore (Karnataka; Southern India) and Salboni (West Bengal; Eastern India). The RBI, along with the Indian Banks' Association, established the National Payments Corporation of India to promote and regulate the payment and settlement systems in India. Deposit Insurance and Credit Guarantee Corporation was established by RBI as one of its specialized division for the purpose of providing insurance of deposits and guarantee

In [21]:
# to setup token usage tracking 
with get_openai_callback() as call:
    response = generate_evaluate_chain(
        {
            'text':TEXT,
            'number':5,
            'subject':'economics',
            'tone':"simple",
            'response_json': json.dumps(RESPONSE_JSON)
        }
    )
    print(call)

e:\Personal\LLM\langchain_mcqgen\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The Reserve Bank of India, abbreviated as RBI, is India's central bank and regulatory body responsible for regulation of the Indian banking system. Owned by the Ministry of Finance, Government of India, it is responsible for the control, issue and maintaining supply of the Indian rupee. It also manages the country's main payment systems and works to promote its economic development. Bharatiya Reserve Bank Note Mudran (BRBNM) is a specialised division of RBI through which it prints and mints Indian currency notes (INR) in two of its currency printing presses located in Mysore (Karnataka; Southern India) and Salboni (West Bengal; Eastern India). The RBI, along with the Indian Banks' Association, established the National Payments Corporation of India to promote and regulate the payment and settlement systems in India. Deposit Insurance and Credit Guarantee Corporation was establish

In [28]:
json.loads(response.get('quiz'))

{'1': {'mcq': 'Who is responsible for the regulation of the Indian banking system?',
  'options': {'a': 'Ministry of Finance',
   'b': 'Reserve Bank of India',
   'c': 'National Payments Corporation of India',
   'd': "Indian Banks' Association"},
  'correct': 'b'},
 '2': {'mcq': 'Where are the two currency printing presses of RBI located?',
  'options': {'a': 'Mumbai and Delhi',
   'b': 'Chennai and Kolkata',
   'c': 'Mysore and Salboni',
   'd': 'Ahmedabad and Jaipur'},
  'correct': 'c'},
 '3': {'mcq': 'When was the Reserve Bank of India nationalized?',
  'options': {'a': '1 April 1935',
   'b': '1 January 1949',
   'c': '15 August 1947',
   'd': '26 January 1950'},
  'correct': 'b'},
 '4': {'mcq': 'Who represents the regional interests in the local boards of RBI?',
  'options': {'a': 'Government-nominated directors',
   'b': 'Deputy governors',
   'c': 'Finance ministry representatives',
   'd': 'Members of co-operative and indigenous banks'},
  'correct': 'd'},
 '5': {'mcq': 'What 

In [29]:
response.get('review')

'The quiz questions are suitable for economics students as they test knowledge on Indian banking system and RBI. The questions are clear and concise, with options covering relevant topics. No changes needed as the complexity matches student abilities.'